In [44]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQA
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Table, Image
from io import BytesIO
import base64
from unstructured.documents.elements import (
    Table,
    Image as USImage,
    CompositeElement,
)
import pytesseract
from PIL import Image as PILImage  # evita conflito de nomes
import os

In [2]:
def get_pdf_text(pdf):
    pieces = []
    reader = PdfReader(pdf)
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        pieces.append(text or '')
        print(f'Página {i+1}:')
        print(text)
    return "".join(pieces)

In [3]:
path = r'C:\Users\Orçamento\Desktop\ENGENHORCA\CAIXA TERMINAL F650.pdf'
raw_text = get_pdf_text(path)
raw_text

Página 1:
Caixas Terminais
F6/1/P/6
Para filtros de alta eficiência, montagem no 
teto e montagem em parede
®
TROX DO BRASIL LTDA.
Rua Alvarenga, 2025  
05509-005 – São Paulo – SP
+55 (11) 3037-3900
trox-br@troxgroup.com
www.troxbrasil.com.br
 +55 11 97395 1627

Página 2:
Índice - Conteúdo - Tipos de produtos
Caixa Terminal F631
Caixa Terminal F640 
(na figura com difusor tipo VDW 676X54)
Caixa Terminal F660 
(na figura com difusor tipo FD)
Caixa Terminal F650
(na figura com difusor tipo ADLQ-A)
Caixa Terminal F670 
(na figura com grelha tipo AT-A)
Índice - Conteúdo -Tipos de produtos ........................... 2
Caixa Terminal F631 com Filtros de Alta Eficiência para 
montagem no teto .......................................................... 3
Caixa Terminal F650 com Filtros de Alta Eficiência para 
montagem no teto .......................................................... 4
Caixa Terminal F640 com Filtros de Alta Eficiência para 
montagem no teto .................................

'Caixas Terminais\nF6/1/P/6\nPara filtros de alta eficiência, montagem no \nteto e montagem em parede\n®\nTROX DO BRASIL LTDA.\nRua Alvarenga, 2025  \n05509-005 – São Paulo – SP\n+55 (11) 3037-3900\ntrox-br@troxgroup.com\nwww.troxbrasil.com.br\n\uf232 +55 11 97395 1627\nÍndice - Conteúdo - Tipos de produtos\nCaixa Terminal F631\nCaixa Terminal F640 \n(na figura com difusor tipo VDW 676X54)\nCaixa Terminal F660 \n(na figura com difusor tipo FD)\nCaixa Terminal F650\n(na figura com difusor tipo ADLQ-A)\nCaixa Terminal F670 \n(na figura com grelha tipo AT-A)\nÍndice - Conteúdo -Tipos de produtos ........................... 2\nCaixa Terminal F631 com Filtros de Alta Eficiência para \nmontagem no teto .......................................................... 3\nCaixa Terminal F650 com Filtros de Alta Eficiência para \nmontagem no teto .......................................................... 4\nCaixa Terminal F640 com Filtros de Alta Eficiência para \nmontagem no teto ....................

In [4]:
def get_text_chunks(raw_text):
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " ", ""],chunk_size=300,chunk_overlap=50)
    texts = text_splitter.split_text(raw_text or '')
    return texts

In [5]:
text_chunks = get_text_chunks(raw_text)
text_chunks

['Caixas Terminais\nF6/1/P/6\nPara filtros de alta eficiência, montagem no \nteto e montagem em parede\n®\nTROX DO BRASIL LTDA.\nRua Alvarenga, 2025  \n05509-005 – São Paulo – SP\n+55 (11) 3037-3900\ntrox-br@troxgroup.com\nwww.troxbrasil.com.br\n\uf232 +55 11 97395 1627\nÍndice - Conteúdo - Tipos de produtos',
 'Índice - Conteúdo - Tipos de produtos\nCaixa Terminal F631\nCaixa Terminal F640 \n(na figura com difusor tipo VDW 676X54)\nCaixa Terminal F660 \n(na figura com difusor tipo FD)\nCaixa Terminal F650\n(na figura com difusor tipo ADLQ-A)\nCaixa Terminal F670 \n(na figura com grelha tipo AT-A)',
 '(na figura com grelha tipo AT-A)\nÍndice - Conteúdo -Tipos de produtos ........................... 2\nCaixa Terminal F631 com Filtros de Alta Eficiência para \nmontagem no teto .......................................................... 3\nCaixa Terminal F650 com Filtros de Alta Eficiência para',
 'montagem no teto .......................................................... 4\nCaixa Termina

In [6]:
def get_vectorstore(text_chunks):
    embeddings = HuggingFaceEmbeddings(model_name='multi-qa-mpnet-base-dot-v1')
    vectorstore = FAISS.from_texts(texts=text_chunks,embedding=embeddings)
    return vectorstore

In [7]:
vectorstore = get_vectorstore(text_chunks)
print(vectorstore)

C:\Users\Orçamento\AppData\Local\Temp\ipykernel_25204\668656990.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='multi-qa-mpnet-base-dot-v1')
c:\Users\Orçamento\Desktop\ENGENHORCA\.venv-pipeline-v3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def get_convesation_chain():
    llm = ChatGroq(model='llama-3.1-8b-instant',temperature=0.2)
    
    prompt = ChatPromptTemplate.from_messages([
        ('system',"Você é um assistente na área de HVAC (ar condicionado). Responda de forma objetiva e correta."),
        MessagesPlaceholder('history'),
        ('human','{question}')
        ])
    
    chain = prompt | llm | StrOutputParser()
    
    return chain

In [9]:
chain = get_convesation_chain()
resposta = chain.invoke({'question':'O que é um split?', 'history':[]})
print(resposta)

Um split é um tipo de sistema de ar condicionado que consiste em dois componentes principais: o condensador e o evaporador.

O condensador é o componente externo, geralmente instalado no exterior da edificação, e é responsável por dissipar o calor do refrigerante. Ele é conectado ao evaporador por meio de um tubo de condensação.

O evaporador, por outro lado, é o componente interno, instalado dentro da edificação, e é responsável por absorver o calor do ambiente e resfriar o ar. Ele é conectado ao condensador por meio do tubo de condensação.

O nome "split" vem do fato de que o sistema é dividido em dois componentes separados, um externo e outro interno, que são conectados por meio de um tubo de condensação. Isso permite uma instalação mais flexível e fácil de manter, pois os componentes podem ser instalados em locais diferentes.

Os sistemas de split são muito comuns em residências e pequenas edificações, pois são fáceis de instalar e manter, e oferecem uma boa relação custo-benefício

In [10]:
retriever = vectorstore.as_retriever(search_kwargs={'k':3})

In [11]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatGroq(model='llama-3.1-8b-instant', temperature=0.2),
    retriever=retriever,
    chain_type='stuff'
)

In [12]:
resposta_documento = qa_chain.invoke({'query': 'O que é uma caixa terminal?'})
print(resposta_documento)

{'query': 'O que é uma caixa terminal?', 'result': 'Com base no contexto fornecido, parece que uma caixa terminal é um tipo de equipamento utilizado em sistemas de ventilação ou ar condicionado. Ela é projetada para ser montada no teto e provavelmente serve como um terminal ou ponto de saída para o ar tratado ou condicionado.\n\nA presença de filtros de alta eficiência e diferentes tipos de difusores ou grelhas sugere que a caixa terminal é projetada para fornecer um fluxo de ar limpo e controlado para a área circundante.'}


In [13]:
def _format_docs(docs):
    # junta os trechos retornados pelo retriever
    return "\n\n---\n\n".join(d.page_content for d in docs)

In [14]:
def build_rag_chain():
    llm = ChatGroq(model='llama-3.1-8b-instant',temperature=0.2)
    
    prompt = ChatPromptTemplate.from_messages([
        ('system',
        "Você é um assistente técnico de HVAC. Responda com base ESTRITA no contexto. "
        "Se não houver contexto suficiente, diga que não encontrou no documento."),
        ('human','Pergunta:{question}\n\n'
        'Contexto (trechos recuperados):\n{context}')
        ])
    return prompt | llm | StrOutputParser()

In [15]:
def handler_user_input(question: str):
    global rag, retriever
    if retriever:
        chain = {
            'context': retriever | RunnableLambda(_format_docs),
            'question': RunnablePassthrough()
            } | build_rag_chain
        answer = chain.invoke(question)
        return answer
    else:
        # fallback: sem contexto
        answer = rag.invoke({"question": question, "context": "(nenhum contexto: nenhum índice carregado)"})
        return answer



In [16]:
print(handler_user_input("O que é um sistema split?"))

TypeError: unsupported operand type(s) for |: 'dict' and 'function'

In [17]:
def format_docs(docs):
    return "\n\n".join(getattr(d, "page_content", str(d)) for d in docs)

In [18]:
def get_conversation_chain_rag(retriever):
    llm = ChatGroq(model='llama-3.1-8b-instant', temperature=0.2)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        "Você é um assistente HVAC. Responda **apenas** com base no contexto.\n\n"
        "Contexto:\n{context}"),
        MessagesPlaceholder("history"),
        ("human", "{question}")
    ])

    chain = {
        "question": RunnablePassthrough(),                       # passa a pergunta
        "history": itemgetter("history"),                        # se você tiver histórico
        "context": itemgetter("question")                        # usa a pergunta
                | retriever                                   # busca no índice
                | RunnableLambda(format_docs),                # formata os docs
    } | prompt | llm | StrOutputParser()

    return chain

In [26]:
user_question = 'Me fale tudo sobre caixa terminal'
chain_rag = get_conversation_chain_rag(retriever)
resp = chain_rag.invoke({"question": user_question, "history": []})

In [27]:
resp

'**Caixa Terminal**\n\nA caixa terminal é um componente fundamental em sistemas de ar condicionado (HVAC) e ventilação. Ela é responsável por distribuir o ar condicionado ou o ar quente para diferentes áreas de um ambiente, garantindo uma distribuição uniforme e eficiente.\n\n**Tipos de Caixa Terminal**\n\nExistem vários tipos de caixa terminal, cada um com suas características e aplicações específicas. Alguns dos principos tipos incluem:\n\n1. **Caixa Terminal F631**: Ideal para montagem no teto, com filtros de alta eficiência e compatível com difusores tipo VDW 676X54.\n2. **Caixa Terminal F640**: Também para montagem no teto, com filtros de alta eficiência e compatível com difusores tipo AT-A.\n3. **Caixa Terminal F650**: Outra opção para montagem no teto, com filtros de alta eficiência e compatível com difusores tipo ADLQ-A.\n4. **Caixa Terminal F660**: Destinado para montagem no teto, com filtros de alta eficiência e compatível com difusores tipo FD.\n5. **Caixa Terminal F670**: P

Com base no contexto fornecido, parece que uma caixa terminal é um tipo de equipamento utilizado em sistemas de ventilação ou ar condicionado. Ela é projetada para ser montada no teto e provavelmente serve como um terminal ou ponto de saída para o ar tratado ou condicionado.\n\nA presença de filtros de alta eficiência e diferentes tipos de difusores ou grelhas sugere que a caixa terminal é projetada para fornecer um fluxo de ar limpo e controlado para a área circundante.

A caixa terminal, também conhecida como unidade terminal ou unidade de superfície, é uma parte fundamental de um sistema de ar condicionado (HVAC). Ela é responsável por fornecer o ar condicionado (ar quente ou frio) para um ambiente específico.\n\nA caixa terminal é composta por um conjunto de componentes, incluindo:\n\n- Um evaporador (ou condensador) para resfriar ou aquecer o ar;\n- Um ventilador para distribuir o ar condicionado;\n- Um filtro para limpar o ar;\n- Um sistema de controle para regular a temperatura e a umidade.\n\nA caixa terminal pode ser instalada em uma parede, no teto ou no chão, dependendo do tipo de sistema de ar condicionado e do projeto de instalação. Ela é conectada à unidade de compressão (ou unidade de condensação) por meio de tubos de condensação e de evaporação.\n\nA caixa terminal é responsável por fornecer o ar condicionado de alta qualidade para o ambiente, mantendo a temperatura e a umidade dentro de limites confortáveis. Ela é uma parte crítica do sistema de ar condicionado e deve ser instalada e manutida corretamente para garantir o desempenho eficaz e a segurança do sistema.

O Caixa Terminal F650 é um produto que é montado no teto e é equipado com filtros de alta eficiência.

Caixa Terminal**\n\nA caixa terminal é um componente fundamental em sistemas de ar condicionado (HVAC) e ventilação. Ela é responsável por distribuir o ar condicionado ou o ar quente para diferentes áreas de um ambiente, garantindo uma distribuição uniforme e eficiente.\n\n**Tipos de Caixa Terminal**\n\nExistem vários tipos de caixa terminal, cada um com suas características e aplicações específicas. Alguns dos principos tipos incluem:\n\n1. **Caixa Terminal F631**: Ideal para montagem no teto, com filtros de alta eficiência e compatível com difusores tipo VDW 676X54.\n2. **Caixa Terminal F640**: Também para montagem no teto, com filtros de alta eficiência e compatível com difusores tipo AT-A.\n3. **Caixa Terminal F650**: Outra opção para montagem no teto, com filtros de alta eficiência e compatível com difusores tipo ADLQ-A.\n4. **Caixa Terminal F660**: Destinado para montagem no teto, com filtros de alta eficiência e compatível com difusores tipo FD.\n5. **Caixa Terminal F670**: Para montagem no teto, com filtros de alta eficiência e compatível com difusores tipo AT-A.\n\n**Características**\n\nAs caixas terminais têm várias características importantes, incluindo:\n\n* **Filtros de Alta Eficiência**: Reduzem a presença de partículas no ar condicionado, garantindo uma qualidade do ar melhorada.\n* **Compatibilidade com Difusores**: As caixas terminais são projetadas para funcionar com difusores específicos, garantindo uma distribuição eficiente do ar condicionado.\n* **Montagem no Teto**: As caixas terminais são projetadas para ser montadas no teto, facilitando a instalação e a manutenção.\n\n**Aplicações**\n\nAs caixas terminais são amplamente utilizadas em diferentes ambientes, incluindo:\n\n* **Residências**: Para fornecer ar condicionado ou ar quente para diferentes áreas da casa.\n* **Comércio**: Para manter um ambiente confortável e saudável para os clientes e funcionários.\n* **Indústria**: Para fornecer ar condicionado ou ar quente para áreas de trabalho e processos industriais.\n\nEm resumo, as caixas terminais são componentes fundamentais em sistemas de ar condicionado e ventilação, oferecendo uma distribuição eficiente e uniforme do ar condicionado ou ar quente para diferentes áreas de um ambiente.

In [32]:
def chunks_image(pdf):
    chunks = partition_pdf(
        filename=pdf,
        infer_table_structure=True,            # extract tables
        strategy="hi_res",                     # mandatory to infer tables

        extract_image_block_types=["Image","Table"],   # Add 'Table' to list to extract image of tables
        # image_output_dir_path=output_path,   # if None, images and tables will saved in base64

        extract_image_block_to_payload=True,   # if true, will extract base64 for API usage

        chunking_strategy="basic",          # or 'basic'
        max_characters=5000,                  # defaults to 500
        combine_text_under_n_chars=500,       # defaults to 0
        new_after_n_chars=500,
        languages=["por","eng"],
        ocr_languages=["por","eng"]

        # extract_images_in_pdf=True,          # deprecated
    )
    
    return chunks

In [33]:
chunks = chunks_image(path)

The ocr_languages kwarg will be deprecated in a future version of unstructured. Please use languages instead.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [35]:
print(len(chunks))

14


In [37]:
def extract_elements(chunks):
    tables, texts, images_b64 = [], [], []

    for ch in chunks:
        # Tabela “pura”
        if isinstance(ch, Table):
            tables.append(ch)

        # Tabelas/Imagens aninhadas
        if hasattr(ch.metadata, "orig_elements") and ch.metadata.orig_elements:
            for el in ch.metadata.orig_elements:
                if isinstance(el, Table):
                    tables.append(el)
                # >>> AQUI: checar Image do unstructured, não PIL
                if isinstance(el, USImage) and getattr(el.metadata, "image_base64", None):
                    images_b64.append(el.metadata.image_base64)

        # Texto (CompositeElement)
        if isinstance(ch, CompositeElement):
            # troque para texts.append(ch) se você QUISER o objeto
            texts.append(getattr(ch, "text", ""))

        # (Opcional) imagem “pura” no nível do chunk
        if isinstance(ch, USImage) and getattr(ch.metadata, "image_base64", None):
            images_b64.append(ch.metadata.image_base64)
            
    print(f'O número de chunks é: {len(chunks)}')
    print(f'O número de tabelas é: {len(tables)}')
    print(f'O número de textos é: {len(texts)}')
    print(f'O número de imagens é: {len(images_b64)}')

    return tables, texts, images_b64

In [38]:
tables, text, images_b64 = extract_elements(chunks)

O número de chunks é: 14
O número de tabelas é: 8
O número de textos é: 13
O número de imagens é: 16


In [43]:
print(tables)

[<unstructured.documents.elements.Table object at 0x000001B1CFA9ED90>, <unstructured.documents.elements.Table object at 0x000001B1C6AD5A10>, <unstructured.documents.elements.Table object at 0x000001B1CFA7D450>, <unstructured.documents.elements.Table object at 0x000001B1CFAAC7D0>, <unstructured.documents.elements.Table object at 0x000001B1CFA93810>, <unstructured.documents.elements.Table object at 0x000001B1CFAAD590>, <unstructured.documents.elements.Table object at 0x000001B1CFABC950>, <unstructured.documents.elements.Table object at 0x000001B1CFAC8B90>]


In [41]:
def ocr_from_images_base64(images_b64):
    image_texts = []
    for b64 in images_b64:
        try:
            image_data = base64.b64decode(b64)
            image = PILImage.open(BytesIO(image_data))
            text = pytesseract.image_to_string(image, lang="por+eng")
            image_texts.append(text)
        except Exception as e:
            print(f"❌ Erro ao processar imagem: {e}")
            image_texts.append(text.strip())
    return image_texts

In [42]:
image_text = ocr_from_images_base64(images_b64)
print(image_text)

['', '', '', '', 'Yas if\n: T\nÁ\n', '', '/ Z\nFo\n\na\n\nHE a\n\n', 'ke—\n\n100, |40,\n\n| —\n\n', 'Pontos de\nmedição de\npressão\n\nTipo de filtro:\nMFP\nMFC\n\nOpção de difusor\n(D,V)\n\nOpção de difusor\nem chapa\nperfurada e\ngrelha\n\n(L, R)\n\nOpção de difusor\n(A,B,C)\n\nl_\ny\nyy\n\n101 27\nis\nbe\n\nE\n\ne\n\nis\n\nVmax = 600m°/h)\n\nud\n\n', '27\n\nExEt\no\n\n101\n\nB281\n\nP\n\n', '', 'BC\n\n= 2100 mº/t\n\nmax.\n\n', 'V max, = 600 m/h\n\nmax.\nY wordy\nVmax. = 600 mê/h\n\nmax.\n\nOK\n\n', '', '', '']


In [45]:
# Prompt
prompt_text = """
You are an assistant. If the input is a question, answer it clearly and concisely.
If the input is a piece of text or table, summarize it.

Input:
{element}
"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"),model='llama-3.1-8b-instant',temperature=0.2)
summarize_chain = {"element": lambda x: x} | prompt | llm | StrOutputParser()

In [46]:
import time
from langchain_core.runnables import Runnable
from typing import List

def safe_batch_process(chain: Runnable, data: List[str], batch_size=1, wait_on_error=10):
    results = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        try:
            batch_results = chain.batch(batch, {"max_concurrency": 2})
            results.extend(batch_results)
        except Exception as e:
            print(f"⏱️ Rate limit atingido ou erro: {e}")
            print(f"🔁 Esperando {wait_on_error} segundos para tentar de novo...")
            time.sleep(wait_on_error)
            # tenta o mesmo batch de novo
            try:
                batch_results = chain.batch(batch, {"max_concurrency": 2})
                results.extend(batch_results)
            except Exception as e2:
                print(f"❌ Ainda deu erro: {e2} — pulando esse batch.")
    return results

In [47]:
table_summaries = safe_batch_process(summarize_chain, tables)

In [48]:
table_summaries

["It appears to be a piece of text with product information. \n\nHere's a summary:\n\nThe text lists three products: \n1. Caixa Terminal F631\n2. Caixa Terminal F650\n3. Caixa Terminal F640\n\nAll three products are described as having high-efficiency filters and are designed for ceiling mounting.",
 'The input appears to be a piece of text describing two types of terminal boxes with high-efficiency filters. \n\nThe text mentions two models: \n\n1. F660, which is mounted on the ceiling.\n2. F670, which is mounted on the wall.\n\nThe text is somewhat unclear due to the presence of repeated characters, but it seems to be describing the installation locations of these terminal boxes.',
 'This appears to be a list of measurements. \n\nHere\'s a summary of the list:\n\n- The list contains multiple measurements of dimensions and weights.\n- The measurements are in the format of "weight (dimension)" or "dimension1 x dimension2 x dimension3".\n- There are multiple sets of measurements with dif